In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.svm import SVC

In [ ]:
# Load the dataset
df = pd.read_csv("data/Y_audio_philes_final_features_mk1.csv")

print(df.head())
print(df.info())
# Missing value
print(df.isnull().sum())

In [ ]:
# Feature selection
df_non_feature = ["artist", "country", "language","track_type"]
df_number_feature = df.select_dtypes(include=[np.number]).columns.tolist()
df_number_feature = [c for c in df_number_feature if c not in df_non_feature]
print(df_number_feature)

# Global standardization
scaler = StandardScaler()
df[df_number_feature] = scaler.fit_transform(df[df_number_feature])

# LabelEncoder
X_df = df[df_number_feature].copy()
le = LabelEncoder()
y = le.fit_transform(df["language"].astype(str))
label_names = le.classes_

In [ ]:
# track_type_group
track_type_all = {0: "complete_song", 1: "vocal_only", 2: "no_vocal"}
track_type_specs = {name: (df["track_type"] == code) for code, name in track_type_all.items()}
for name, mask in track_type_specs.items():
    print(name, ":", int(mask.sum()))

In [ ]:
# k-Fold Cross-Validation
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [ ]:
# Model training
models = {
    "LogReg": LogisticRegression(max_iter=500, class_weight="balanced", random_state=42),
    "RandomForest": RandomForestClassifier(
        n_estimators=400, class_weight="balanced_subsample", random_state=42, n_jobs=-1
    ),
    "SVM_linear": SVC(kernel="linear", class_weight="balanced", random_state=42),
}

scoring = {
    "accuracy": "accuracy",
    "precision": "precision_macro",
    "recall": "recall_macro",
    "f1": "f1_macro",
}

rows = []

# Evaluation
for track_type_code, ablation_name in track_type_all.items():
    mask = df["track_type"] == track_type_code
    df_eval = df.loc[mask].reset_index(drop=True)

    X = df_eval[df_number_feature].copy()
    y = le.transform(df_eval["language"].astype(str))

    for name, clf in models.items():
        # Use skf and n_jobs=1 here to avoid TerminatedWorkerError
        cv = cross_validate(clf, X, y, cv=skf, scoring=scoring, n_jobs=1, return_train_score=False)
        acc = np.mean(cv["test_accuracy"])
        prec = np.mean(cv["test_precision"])
        rec = np.mean(cv["test_recall"])
        f1 = np.mean(cv["test_f1"])
        rows.append(
            {"ablation": ablation_name, "model": name, "acc": acc, "prec": prec, "rec": rec, "f1": f1}
        )
        print(f"[{ablation_name} - {name}] Acc={acc:.3f} | Prec={prec:.3f} | Rec={rec:.3f} | F1={f1:.3f}")

In [ ]:
# Final selection
chosen_ablation = "vocal_only"    
chosen_model    = "SVM_linear"   
print("[Final model]", "Ablation =", chosen_ablation, "| Model =", chosen_model)

df_best = df.loc[track_type_specs[chosen_ablation]].reset_index(drop=True) 
X_best = df_best[df_number_feature].copy()
y_best = le.transform(df_best["language"].astype(str))

final_clf = SVC(kernel="linear", class_weight="balanced", random_state=42)

final_clf.fit(X_best, y_best)

print("[Final Train] Done on", chosen_ablation, "-", chosen_model, "| n_samples =", len(df_best))